## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,68.00,94,20,3.44,few clouds
1,1,Avarua,CK,-21.2078,-159.7750,77.00,100,20,3.44,few clouds
2,2,Albanel,CA,48.8832,-72.4487,55.40,50,97,6.91,overcast clouds
3,3,Buraydah,SA,26.3260,43.9750,84.20,26,0,17.27,clear sky
4,4,Mataura,NZ,-46.1927,168.8643,55.99,92,97,1.99,overcast clouds


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avarua,CK,-21.2078,-159.7750,77.00,100,20,3.44,few clouds
3,3,Buraydah,SA,26.3260,43.9750,84.20,26,0,17.27,clear sky
14,14,Bambous Virieux,MU,-20.3428,57.7575,80.01,78,40,11.50,scattered clouds
17,17,Atuona,PF,-9.8000,-139.0333,80.40,77,75,18.72,light rain
34,34,Beira,MZ,-19.8436,34.8389,78.80,78,20,9.28,few clouds
35,35,Kloulklubed,PW,7.0419,134.2556,77.00,94,75,10.27,broken clouds
37,37,Rikitea,PF,-23.1203,-134.9692,78.33,72,2,9.35,clear sky
40,40,Morondava,MG,-20.2833,44.2833,80.28,73,60,10.33,broken clouds
50,50,Vanimo,PG,-2.6741,141.3028,76.71,90,100,6.24,overcast clouds
52,52,Castro,BR,-24.7911,-50.0119,78.62,38,1,1.59,clear sky


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                205
City                   205
Country                202
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows. 
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                202
City                   202
Country                202
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Current Description    202
dtype: int64

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,77.00,few clouds,-21.2078,-159.7750,
3,Buraydah,SA,84.20,clear sky,26.3260,43.9750,
14,Bambous Virieux,MU,80.01,scattered clouds,-20.3428,57.7575,
17,Atuona,PF,80.40,light rain,-9.8000,-139.0333,
34,Beira,MZ,78.80,few clouds,-19.8436,34.8389,
35,Kloulklubed,PW,77.00,broken clouds,7.0419,134.2556,
37,Rikitea,PF,78.33,clear sky,-23.1203,-134.9692,
40,Morondava,MG,80.28,broken clouds,-20.2833,44.2833,
50,Vanimo,PG,76.71,overcast clouds,-2.6741,141.3028,
52,Castro,BR,78.62,clear sky,-24.7911,-50.0119,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,77.00,few clouds,-21.2078,-159.7750,Paradise Inn
3,Buraydah,SA,84.20,clear sky,26.3260,43.9750,فندق راحة السلام
14,Bambous Virieux,MU,80.01,scattered clouds,-20.3428,57.7575,Casa Tia Villa
17,Atuona,PF,80.40,light rain,-9.8000,-139.0333,Villa Enata
34,Beira,MZ,78.80,few clouds,-19.8436,34.8389,Beira Terrace Hotel
35,Kloulklubed,PW,77.00,broken clouds,7.0419,134.2556,Storyboard Beach Resort
37,Rikitea,PF,78.33,clear sky,-23.1203,-134.9692,Pension Maro'i
40,Morondava,MG,80.28,broken clouds,-20.2833,44.2833,La Case Bambou
50,Vanimo,PG,76.71,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
52,Castro,BR,78.62,clear sky,-24.7911,-50.0119,CHACARA BAILLY


In [36]:
# 7 Drop any rows in the hotel_df DataFram where a hotel name is not found.
hotel_df.count()

City                   202
Country                202
Max Temp               202
Current Description    202
Lat                    202
Lng                    202
Hotel Name             202
dtype: int64

In [41]:
# Hotel Name has same count as other columns, therefore hotel_df.dropna() not required.
clean_hotel_df = hotel_df

In [42]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [4]:
# Recall output file,i.e. re-use.
clean_hotel_df = pd.read_csv("WeatherPy_vacation.csv")

In [5]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl> 
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [8]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))